In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import re
import numpy as np
import torch
from sklearn.impute import SimpleImputer
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class fingerprints(nn.Module):
    def __init__(self):
        super(fingerprints, self).__init__()
        self.fc1 = nn.Linear(325,204) # 全连接层
        #print(self.fc1.weight.shape)
        self.fc2 = nn.Linear(204,13963) # 全连接层
        self.fc3 = nn.Linear(13963,11230) # 全连接层
        self.fc4 = nn.Linear(11230,20861) # 全连接层
        self.fc5 = nn.Linear(20861,4608) # 全连接层
        self.fc6 = nn.Linear(4608,1) # 全连接层

    def forward(self, x):
        #print("0:"+ str(x.size())) # 输出tensor大小
        x = F.relu(self.fc1(x))
        #print("1:"+ str(x.size())) # 输出tensor大小
        x = F.relu(self.fc2(x))
        #print("2:"+ str(x.size())) # 输出tensor大小
        x = F.relu(self.fc3(x))
        #print("3:"+ str(x1.size())) # 输出tensor大小
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        #x = F.relu(self.fc6(x))
        x = F.softplus(self.fc6(x))
        #prelu = torch.nn.PReLU(num_parameters=1)
        return x

In [3]:
import pandas as pd
import sys
import time
data = pd.read_csv("data.csv",sep=",")
Y = pd.read_csv("Y_1.csv",sep=",")
Y.iloc[0]

2800    1.000000
4429    0.578947
3583    0.500000
7915    0.071429
455     0.347826
          ...   
6627    0.107143
6083    0.100000
78      0.347826
1063    0.320000
3298    0.259259
Name: 0, Length: 379, dtype: float64

In [4]:
#n_epochs = 200 # 模型训练20轮
log_interval = 210 #控制打印频率的，设n = 210*batch_size，即n张图后打印一次进度
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 根据设备是否支持GPU来选择硬件 
#DEVICE_1 = torch.device("cpu") # 根据设备是否支持GPU来选择硬件 
learn_rate = 0.03 # 学习率
momentum = 0.1  # 动量
network_fingerprints = fingerprints().to(DEVICE)
network_fingerprints.eval()
optimizer_fingerprints = optim.SGD(network_fingerprints.parameters(), lr=learn_rate, momentum=momentum) # 学习率，动量
train_losses = []
train_counter = []

In [5]:
def train(epoch, device):
    network_fingerprints.train() # 调用上一步实例化对象network中的方法（该方法包内已经写好）
    for batch_idx, (data, target) in enumerate(trainloader_fp): # 按batch_size为集合对象进行逐个处理
        data, target = data.to(device), target.to(device) # data是图片，target是标签，device是为了有GPU情况下使用GPU加速
        optimizer_fingerprints.zero_grad() # 开始进行BP之前将梯度设置为零，因为PyTorch会在随后的BP中累积梯度    
        output_fp = network_fingerprints(data) 
        loss_fn = nn.SmoothL1Loss()
        loss2 = loss_fn(output_fp, target)
        #print(loss)
        loss2.backward(retain_graph=True) # 根据误差进行BP
        optimizer_fingerprints.step()
        if batch_idx % log_interval == 0: # 控制输出频率
            print('Train fp Epoch: {} \tLoss: {:.6f}'.format(epoch, loss2.item()))
    #print(hi_fp.shape)
    #print(hi_fv.shape)
    #hi_fp = F.normalize(hi_fp,p=2,dim=0)
    optimizer_fingerprints.step()
    return loss2

In [6]:
from sklearn.metrics import mean_squared_error # 均方误差
from sklearn.metrics import mean_absolute_error # 平方绝对误差
def test(device,n):
    network_fingerprints.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        i = 0
        for data,target in testloader_fp:
            data, target = data.to(device), target.to(device)
            output = network_fingerprints(data)
            loss_fn = nn.SmoothL1Loss()
            out = output.cuda().data.cpu().detach().numpy()
            tar = target.cuda().data.cpu().detach().numpy()
            MSE = mean_squared_error(out, tar)
            MAE = mean_absolute_error(out, tar)
            print('test fp MSE: {:.6f}'.format(MSE))
            print('test fp MAE: {:.6f}'.format(MAE))
            #print(target)
            #print("输出")
            loss = loss_fn(output, target)
            i = i+1
        #print(MSE)
        #print(MAE)
        #print(loss)
        list = [MSE,MAE,loss.cuda().data.cpu().detach().numpy()]
        df = pd.DataFrame(list)
        df.to_csv("%d.csv"%(n))
        print('test fp Loss: {:.6f}'.format(loss.item()))
        return out

In [7]:
from read_file import read_file
from sklearn.model_selection import train_test_split
paras = list(network_fingerprints.parameters())
m1_1 = paras[0].clone()
m2_2 = paras[1].clone()
m3_3 = paras[2].clone()
m4_4 = paras[3].clone()
m5_5 = paras[4].clone()
m6_6 = paras[5].clone()
m7_7 = paras[6].clone()
m8_8 = paras[7].clone()
m9_9 = paras[8].clone()
m10_10 = paras[9].clone()
m11_11 = paras[10].clone()
m12_12 = paras[11].clone()
#m13_13 = paras[12].clone()
X_train,X_test,y_train,y_test = train_test_split(data, Y.iloc[0],test_size=0.2)

In [8]:
list1 = []
list2 = []
for i in range(0,len(y_train.index)):
    #print(y_train.index[i])
    #list.append(int(y_train.index[i]))
    list1.append(y_train.index[i])
for i in range(0,len(y_test.index)):
    #print(y_train.index[i])
    #list.append(int(y_train.index[i]))
    list2.append(y_test.index[i])
ytest=pd.DataFrame(list2)
ytest.to_csv("ytest_tain.csv")
#list

In [9]:
n = 10
h = 20
import torch
from read_file import read_file
from sklearn.model_selection import train_test_split
for i in range(0,n):
    print("第%d个药方"%(i+1))
    y_train = Y[list1].iloc[i]
    y_test = Y[list2].iloc[i]
    trainset_fp = torch.utils.data.TensorDataset(torch.FloatTensor(np.array(X_train)), torch.FloatTensor(np.array(y_train)))
    trainloader_fp = torch.utils.data.DataLoader(trainset_fp, batch_size=len(y_train), shuffle=True, num_workers=2)
    testset_fp = torch.utils.data.TensorDataset(torch.FloatTensor(np.array(X_test)), torch.FloatTensor(np.array(y_test)))
    testloader_fp = torch.utils.data.DataLoader(testset_fp, batch_size=len(y_test), shuffle=True, num_workers=2)
    #备份第一次数据
    print("完成数据切分")
    m1,m2,m3,m4,m5,m6,m7,m8,m9 = read_file(i)
    print("完成数据读取，开始稀疏化网络")
    paras = list(network_fingerprints.parameters())
    with torch.no_grad():
        print("开始处理草药权重矩阵")
        for k in range(0,len(m1)):
            paras[1][k] = m2_2[k]*m1[k][0]
        print("开始处理化合物权重矩阵")
        for k in range(0,len(m3)):
            paras[3][k] = m4_4[k]*m3[k][0]
        print("\n开始处理基因权重矩阵")
        for k in range(0,len(m5)):
            paras[5][k] = m6_6[k]*m5[k][0]
        print("\n开始处理通路权重矩阵")
        for k in range(0,len(m7)):
            paras[7][k] = m8_8[k]*m7[k][0]
        print("\n开始处理功能权重矩阵")
        for k in range(0,len(m9)):
            paras[9][k] = m10_10[k]*m9[k][0]
    print("开始训练和测试")
    import warnings
    warnings.filterwarnings("ignore")
    n_epochs = 200
    loss1 = []
    for epoch in range(1, n_epochs + 1):
        loss = train(epoch, DEVICE)
        aa = (loss.cpu()).detach().numpy()
        loss1.append(aa)
    loss1=pd.DataFrame(loss1)
    loss1.to_csv("%d.csv"%(i))
    #test(DEVICE,i)
    out = test(DEVICE,i)
    df = pd.DataFrame(out)
    df.to_csv("%d.csv"%(h))
    h = h+1
    paras = list(network_fingerprints.parameters())
    print("开始备份网络权重")
    with torch.no_grad():
        print("开始处理草药权重矩阵")
        for k in range(0,paras[1].shape[0]):
            if(paras[1][k] != 0):
                m2_2[k] = paras[1][k].clone()
        print("开始处理化合物权重矩阵")
        for k in range(0,paras[3].shape[0]):
            if(paras[3][k] != 0):
                m4_4[k] = paras[3][k].clone()
        print("\n开始处理基因权重矩阵")
        for k in range(0,paras[5].shape[0]):
            if(paras[5][k] != 0):
                m6_6[k] = paras[5][k].clone()
        print("\n开始处理通路权重矩阵")
        for k in range(0,paras[7].shape[0]):
            if(paras[7][k] != 0):
                m8_8[i] = paras[7][i].clone()
        print("\n开始处理功能权重矩阵")
        for k in range(0,paras[9].shape[0]):
            if(paras[9][k] != 0):
                m10_10[k] = paras[9][k].clone() 

第1个药方
完成数据切分
完成数据读取，开始稀疏化网络
开始处理草药权重矩阵
开始处理化合物权重矩阵

开始处理基因权重矩阵

开始处理通路权重矩阵

开始处理功能权重矩阵
开始训练和测试
Train fp Epoch: 1 	Loss: 0.069379
Train fp Epoch: 2 	Loss: 0.066734
Train fp Epoch: 3 	Loss: 0.064109
Train fp Epoch: 4 	Loss: 0.061652
Train fp Epoch: 5 	Loss: 0.059350
Train fp Epoch: 6 	Loss: 0.057190
Train fp Epoch: 7 	Loss: 0.055159
Train fp Epoch: 8 	Loss: 0.053246
Train fp Epoch: 9 	Loss: 0.051440
Train fp Epoch: 10 	Loss: 0.049733
Train fp Epoch: 11 	Loss: 0.048118
Train fp Epoch: 12 	Loss: 0.046588
Train fp Epoch: 13 	Loss: 0.045136
Train fp Epoch: 14 	Loss: 0.043758
Train fp Epoch: 15 	Loss: 0.042448
Train fp Epoch: 16 	Loss: 0.041203
Train fp Epoch: 17 	Loss: 0.040018
Train fp Epoch: 18 	Loss: 0.038890
Train fp Epoch: 19 	Loss: 0.037815
Train fp Epoch: 20 	Loss: 0.036788
Train fp Epoch: 21 	Loss: 0.035809
Train fp Epoch: 22 	Loss: 0.034872
Train fp Epoch: 23 	Loss: 0.033977
Train fp Epoch: 24 	Loss: 0.033122
Train fp Epoch: 25 	Loss: 0.032302
Train fp Epoch: 26 	Loss: 0.031518
Trai

Train fp Epoch: 24 	Loss: 0.014836
Train fp Epoch: 25 	Loss: 0.014834
Train fp Epoch: 26 	Loss: 0.014832
Train fp Epoch: 27 	Loss: 0.014830
Train fp Epoch: 28 	Loss: 0.014828
Train fp Epoch: 29 	Loss: 0.014826
Train fp Epoch: 30 	Loss: 0.014824
Train fp Epoch: 31 	Loss: 0.014822
Train fp Epoch: 32 	Loss: 0.014820
Train fp Epoch: 33 	Loss: 0.014819
Train fp Epoch: 34 	Loss: 0.014817
Train fp Epoch: 35 	Loss: 0.014815
Train fp Epoch: 36 	Loss: 0.014813
Train fp Epoch: 37 	Loss: 0.014812
Train fp Epoch: 38 	Loss: 0.014810
Train fp Epoch: 39 	Loss: 0.014809
Train fp Epoch: 40 	Loss: 0.014807
Train fp Epoch: 41 	Loss: 0.014806
Train fp Epoch: 42 	Loss: 0.014804
Train fp Epoch: 43 	Loss: 0.014803
Train fp Epoch: 44 	Loss: 0.014801
Train fp Epoch: 45 	Loss: 0.014800
Train fp Epoch: 46 	Loss: 0.014798
Train fp Epoch: 47 	Loss: 0.014797
Train fp Epoch: 48 	Loss: 0.014796
Train fp Epoch: 49 	Loss: 0.014794
Train fp Epoch: 50 	Loss: 0.014793
Train fp Epoch: 51 	Loss: 0.014792
Train fp Epoch: 52 	

Train fp Epoch: 49 	Loss: 0.010039
Train fp Epoch: 50 	Loss: 0.010031
Train fp Epoch: 51 	Loss: 0.010023
Train fp Epoch: 52 	Loss: 0.010015
Train fp Epoch: 53 	Loss: 0.010008
Train fp Epoch: 54 	Loss: 0.010001
Train fp Epoch: 55 	Loss: 0.009995
Train fp Epoch: 56 	Loss: 0.009988
Train fp Epoch: 57 	Loss: 0.009982
Train fp Epoch: 58 	Loss: 0.009976
Train fp Epoch: 59 	Loss: 0.009970
Train fp Epoch: 60 	Loss: 0.009965
Train fp Epoch: 61 	Loss: 0.009959
Train fp Epoch: 62 	Loss: 0.009954
Train fp Epoch: 63 	Loss: 0.009949
Train fp Epoch: 64 	Loss: 0.009945
Train fp Epoch: 65 	Loss: 0.009940
Train fp Epoch: 66 	Loss: 0.009936
Train fp Epoch: 67 	Loss: 0.009932
Train fp Epoch: 68 	Loss: 0.009928
Train fp Epoch: 69 	Loss: 0.009924
Train fp Epoch: 70 	Loss: 0.009920
Train fp Epoch: 71 	Loss: 0.009916
Train fp Epoch: 72 	Loss: 0.009913
Train fp Epoch: 73 	Loss: 0.009909
Train fp Epoch: 74 	Loss: 0.009906
Train fp Epoch: 75 	Loss: 0.009903
Train fp Epoch: 76 	Loss: 0.009900
Train fp Epoch: 77 	

Train fp Epoch: 74 	Loss: 0.006146
Train fp Epoch: 75 	Loss: 0.006083
Train fp Epoch: 76 	Loss: 0.006023
Train fp Epoch: 77 	Loss: 0.005966
Train fp Epoch: 78 	Loss: 0.005910
Train fp Epoch: 79 	Loss: 0.005856
Train fp Epoch: 80 	Loss: 0.005804
Train fp Epoch: 81 	Loss: 0.005755
Train fp Epoch: 82 	Loss: 0.005706
Train fp Epoch: 83 	Loss: 0.005660
Train fp Epoch: 84 	Loss: 0.005615
Train fp Epoch: 85 	Loss: 0.005571
Train fp Epoch: 86 	Loss: 0.005529
Train fp Epoch: 87 	Loss: 0.005489
Train fp Epoch: 88 	Loss: 0.005450
Train fp Epoch: 89 	Loss: 0.005412
Train fp Epoch: 90 	Loss: 0.005375
Train fp Epoch: 91 	Loss: 0.005340
Train fp Epoch: 92 	Loss: 0.005305
Train fp Epoch: 93 	Loss: 0.005272
Train fp Epoch: 94 	Loss: 0.005240
Train fp Epoch: 95 	Loss: 0.005208
Train fp Epoch: 96 	Loss: 0.005178
Train fp Epoch: 97 	Loss: 0.005149
Train fp Epoch: 98 	Loss: 0.005121
Train fp Epoch: 99 	Loss: 0.005093
Train fp Epoch: 100 	Loss: 0.005066
Train fp Epoch: 101 	Loss: 0.005040
Train fp Epoch: 10

Train fp Epoch: 99 	Loss: 0.004316
Train fp Epoch: 100 	Loss: 0.004305
Train fp Epoch: 101 	Loss: 0.004293
Train fp Epoch: 102 	Loss: 0.004282
Train fp Epoch: 103 	Loss: 0.004272
Train fp Epoch: 104 	Loss: 0.004262
Train fp Epoch: 105 	Loss: 0.004252
Train fp Epoch: 106 	Loss: 0.004243
Train fp Epoch: 107 	Loss: 0.004234
Train fp Epoch: 108 	Loss: 0.004225
Train fp Epoch: 109 	Loss: 0.004217
Train fp Epoch: 110 	Loss: 0.004209
Train fp Epoch: 111 	Loss: 0.004202
Train fp Epoch: 112 	Loss: 0.004194
Train fp Epoch: 113 	Loss: 0.004187
Train fp Epoch: 114 	Loss: 0.004181
Train fp Epoch: 115 	Loss: 0.004174
Train fp Epoch: 116 	Loss: 0.004168
Train fp Epoch: 117 	Loss: 0.004162
Train fp Epoch: 118 	Loss: 0.004157
Train fp Epoch: 119 	Loss: 0.004151
Train fp Epoch: 120 	Loss: 0.004146
Train fp Epoch: 121 	Loss: 0.004141
Train fp Epoch: 122 	Loss: 0.004136
Train fp Epoch: 123 	Loss: 0.004131
Train fp Epoch: 124 	Loss: 0.004127
Train fp Epoch: 125 	Loss: 0.004123
Train fp Epoch: 126 	Loss: 0.

Train fp Epoch: 124 	Loss: 0.004345
Train fp Epoch: 125 	Loss: 0.004345
Train fp Epoch: 126 	Loss: 0.004345
Train fp Epoch: 127 	Loss: 0.004345
Train fp Epoch: 128 	Loss: 0.004345
Train fp Epoch: 129 	Loss: 0.004344
Train fp Epoch: 130 	Loss: 0.004344
Train fp Epoch: 131 	Loss: 0.004344
Train fp Epoch: 132 	Loss: 0.004344
Train fp Epoch: 133 	Loss: 0.004344
Train fp Epoch: 134 	Loss: 0.004343
Train fp Epoch: 135 	Loss: 0.004343
Train fp Epoch: 136 	Loss: 0.004343
Train fp Epoch: 137 	Loss: 0.004343
Train fp Epoch: 138 	Loss: 0.004343
Train fp Epoch: 139 	Loss: 0.004343
Train fp Epoch: 140 	Loss: 0.004342
Train fp Epoch: 141 	Loss: 0.004342
Train fp Epoch: 142 	Loss: 0.004342
Train fp Epoch: 143 	Loss: 0.004342
Train fp Epoch: 144 	Loss: 0.004342
Train fp Epoch: 145 	Loss: 0.004341
Train fp Epoch: 146 	Loss: 0.004341
Train fp Epoch: 147 	Loss: 0.004341
Train fp Epoch: 148 	Loss: 0.004341
Train fp Epoch: 149 	Loss: 0.004341
Train fp Epoch: 150 	Loss: 0.004341
Train fp Epoch: 151 	Loss: 0

Train fp Epoch: 149 	Loss: 0.011177
Train fp Epoch: 150 	Loss: 0.011176
Train fp Epoch: 151 	Loss: 0.011176
Train fp Epoch: 152 	Loss: 0.011175
Train fp Epoch: 153 	Loss: 0.011174
Train fp Epoch: 154 	Loss: 0.011174
Train fp Epoch: 155 	Loss: 0.011173
Train fp Epoch: 156 	Loss: 0.011172
Train fp Epoch: 157 	Loss: 0.011172
Train fp Epoch: 158 	Loss: 0.011171
Train fp Epoch: 159 	Loss: 0.011171
Train fp Epoch: 160 	Loss: 0.011170
Train fp Epoch: 161 	Loss: 0.011170
Train fp Epoch: 162 	Loss: 0.011169
Train fp Epoch: 163 	Loss: 0.011169
Train fp Epoch: 164 	Loss: 0.011168
Train fp Epoch: 165 	Loss: 0.011168
Train fp Epoch: 166 	Loss: 0.011167
Train fp Epoch: 167 	Loss: 0.011167
Train fp Epoch: 168 	Loss: 0.011167
Train fp Epoch: 169 	Loss: 0.011166
Train fp Epoch: 170 	Loss: 0.011166
Train fp Epoch: 171 	Loss: 0.011165
Train fp Epoch: 172 	Loss: 0.011165
Train fp Epoch: 173 	Loss: 0.011165
Train fp Epoch: 174 	Loss: 0.011164
Train fp Epoch: 175 	Loss: 0.011164
Train fp Epoch: 176 	Loss: 0

Train fp Epoch: 174 	Loss: 0.018407
Train fp Epoch: 175 	Loss: 0.018407
Train fp Epoch: 176 	Loss: 0.018407
Train fp Epoch: 177 	Loss: 0.018407
Train fp Epoch: 178 	Loss: 0.018407
Train fp Epoch: 179 	Loss: 0.018407
Train fp Epoch: 180 	Loss: 0.018406
Train fp Epoch: 181 	Loss: 0.018406
Train fp Epoch: 182 	Loss: 0.018406
Train fp Epoch: 183 	Loss: 0.018406
Train fp Epoch: 184 	Loss: 0.018406
Train fp Epoch: 185 	Loss: 0.018406
Train fp Epoch: 186 	Loss: 0.018406
Train fp Epoch: 187 	Loss: 0.018405
Train fp Epoch: 188 	Loss: 0.018405
Train fp Epoch: 189 	Loss: 0.018405
Train fp Epoch: 190 	Loss: 0.018405
Train fp Epoch: 191 	Loss: 0.018405
Train fp Epoch: 192 	Loss: 0.018405
Train fp Epoch: 193 	Loss: 0.018405
Train fp Epoch: 194 	Loss: 0.018404
Train fp Epoch: 195 	Loss: 0.018404
Train fp Epoch: 196 	Loss: 0.018404
Train fp Epoch: 197 	Loss: 0.018404
Train fp Epoch: 198 	Loss: 0.018404
Train fp Epoch: 199 	Loss: 0.018404
Train fp Epoch: 200 	Loss: 0.018404
test fp MSE: 0.038428
test f

Train fp Epoch: 199 	Loss: 0.003800
Train fp Epoch: 200 	Loss: 0.003799
test fp MSE: 0.003552
test fp MAE: 0.048080
test fp Loss: 0.001688
开始备份网络权重
开始处理草药权重矩阵
开始处理化合物权重矩阵

开始处理基因权重矩阵

开始处理通路权重矩阵

开始处理功能权重矩阵
第10个药方
完成数据切分
完成数据读取，开始稀疏化网络
开始处理草药权重矩阵
开始处理化合物权重矩阵

开始处理基因权重矩阵

开始处理通路权重矩阵

开始处理功能权重矩阵
开始训练和测试
Train fp Epoch: 1 	Loss: 0.069109
Train fp Epoch: 2 	Loss: 0.067454
Train fp Epoch: 3 	Loss: 0.065729
Train fp Epoch: 4 	Loss: 0.064041
Train fp Epoch: 5 	Loss: 0.062395
Train fp Epoch: 6 	Loss: 0.060795
Train fp Epoch: 7 	Loss: 0.059244
Train fp Epoch: 8 	Loss: 0.057744
Train fp Epoch: 9 	Loss: 0.056297
Train fp Epoch: 10 	Loss: 0.054904
Train fp Epoch: 11 	Loss: 0.053567
Train fp Epoch: 12 	Loss: 0.052286
Train fp Epoch: 13 	Loss: 0.051058
Train fp Epoch: 14 	Loss: 0.049885
Train fp Epoch: 15 	Loss: 0.048764
Train fp Epoch: 16 	Loss: 0.047694
Train fp Epoch: 17 	Loss: 0.046674
Train fp Epoch: 18 	Loss: 0.045703
Train fp Epoch: 19 	Loss: 0.044777
Train fp Epoch: 20 	Loss: 0.043895
Train 

In [10]:
paras = list(network_fingerprints.parameters())
paras

[Parameter containing:
 tensor([[-0.0169, -0.0195,  0.0229,  ..., -0.0049, -0.0282,  0.0212],
         [ 0.0327, -0.0333,  0.0539,  ..., -0.0380, -0.0376, -0.0410],
         [ 0.0223,  0.0275, -0.0289,  ...,  0.0388,  0.0146,  0.0042],
         ...,
         [-0.0320,  0.0273, -0.0006,  ..., -0.0026, -0.0479,  0.0214],
         [ 0.0129,  0.0027,  0.0276,  ..., -0.0500,  0.0328,  0.0517],
         [ 0.0239, -0.0281,  0.0201,  ..., -0.0458, -0.0004,  0.0084]],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([-1.6281e-04, -3.7707e-04, -3.8406e-03, -1.4018e-04, -6.5165e-05,
         -2.8997e-04, -7.9929e-05, -1.5069e-04, -1.5647e-03, -2.1975e-05,
         -3.8317e-04,  2.0882e-06, -6.7789e-05, -7.9984e-05, -1.0883e-05,
         -6.6517e-03, -1.1587e-05, -7.4261e-06, -9.6829e-04, -9.7515e-03,
         -2.7488e-03, -1.5591e-04, -1.2686e-04, -6.3524e-03, -3.9933e-04,
         -3.9027e-03, -5.0979e-03, -1.7586e-03, -8.2502e-04, -4.9332e-03,
         -1.0107e-02, -

In [11]:
m2_2

tensor([-1.6281e-04, -3.7707e-04, -3.8406e-03, -1.4018e-04, -6.5165e-05,
        -2.8997e-04, -7.9929e-05, -1.5069e-04, -1.5647e-03, -2.1975e-05,
        -3.8317e-04,  2.0882e-06, -6.7789e-05, -7.9984e-05, -1.0883e-05,
        -6.6517e-03, -1.1587e-05, -7.4261e-06, -9.6829e-04, -9.7515e-03,
        -2.7488e-03, -1.5591e-04, -1.2686e-04, -6.3524e-03, -3.9933e-04,
        -3.9027e-03, -5.0979e-03, -1.7586e-03, -8.2502e-04, -4.9332e-03,
        -1.0107e-02, -2.2639e-03,  3.4629e-04, -1.0256e-03, -6.5801e-04,
        -7.8543e-04, -3.8927e-03, -2.0791e-03, -1.3868e-03,  1.8713e-04,
        -3.6854e-03, -2.4888e-03,  2.4194e-04, -4.0253e-03, -2.1362e-04,
        -3.4350e-04, -9.4776e-06, -3.2689e-04, -8.8107e-03, -1.4375e-03,
        -6.0070e-05, -1.4061e-03, -8.0225e-03, -3.0136e-04, -1.8975e-03,
        -1.9388e-03, -5.1097e-04, -1.6307e-06, -5.5736e-04, -9.1474e-04,
        -7.2970e-04, -3.9461e-03, -8.4851e-03, -1.8618e-04, -1.4274e-04,
         1.6450e-04, -1.3294e-04, -4.8222e-03, -4.4

In [12]:
 with torch.no_grad():
    print("开始处理草药权重矩阵")
    for i in range(0,paras[1].shape[0]):
        paras[1][i] = m2_2[i]
    print("开始处理化合物权重矩阵")
    for i in range(0,paras[3].shape[0]):
        paras[3][i] = m4_4[i]
    print("\n开始处理基因权重矩阵")
    for i in range(0,paras[5].shape[0]):
        paras[5][i] = m6_6[i]
    print("\n开始处理通路权重矩阵")
    for i in range(0,paras[7].shape[0]):
        paras[7][i] = m8_8[i]
    print("\n开始处理功能权重矩阵")
    for i in range(0,paras[9].shape[0]):
        paras[9][i] = m10_10[i]

开始处理草药权重矩阵
开始处理化合物权重矩阵

开始处理基因权重矩阵

开始处理通路权重矩阵

开始处理功能权重矩阵


In [13]:
torch.save(network_fingerprints, "H:\工作记录\中医药\数据集2\entire_model.pt")